<a href="https://colab.research.google.com/github/kschmi68/NLP_2024/blob/main/Copy_of_002_Alto_XML_Dateien_einlesen_und_Texte_extrahieren.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summer School: Digitale Methoden der Zeitungsanalyse

## Teil 2: Alto-XML-Dateien einlesen und Text extrahieren

In diesem Teil werden zunächst die XML-Dateien eingelesen und der reine Text extrahiert.

Um die Arbeitsumgebung für die folgenden Schritte passend einzurichten, sollten zunächst die benötigten Python-Biblitoheken importiert werden.

- `pandas`: Bibliothek zur Datenanalyse.
- `lxml`: Bibliothek zur schnellen und flexiblen Verarbeitung von XML- und HTML-Dokumenten
- `BeautifulSoup`: Bibliothek zum einfachen Parsen und Scrapen von HTML- und XML-Dokumenten
- `unicodedata`: Standardbibliothek zur Handhabung und Normalisierung von Unicode-Daten
- `pathlib`: Biobliothek zur Arbeit mit Dateisystempfaden in Python
- `tqdm`: Zur Erstellung von Fortschrittsbalken in Python-Schleifen



In [1]:
import pandas as pd
from lxml import etree
from tqdm import tqdm
import unicodedata
from bs4 import BeautifulSoup
from pathlib import Path

Optional: Lokale Installation

In [2]:
%pip install -q tqdm
%pip install -q bs4
%pip install -q pathlib
%pip install -q matplotlib
%pip install -q numpy
%pip install -q wordcloud
%pip install -q plotly.express
%pip install -q spacy
%pip install -q nltk

#### Exemplarisches Laden **einer** XML-Datei:

Direktes laden einer Datei:

In [3]:
with open ("La_Otra_Alemania/1942-01-01_BWSVNEKFTM7SPW4SQAWQQNGIHHTRFDS4/DDB_FULLTEXT/5.xml") as f:
    content = f.read()

FileNotFoundError: [Errno 2] No such file or directory: 'La_Otra_Alemania/1942-01-01_BWSVNEKFTM7SPW4SQAWQQNGIHHTRFDS4/DDB_FULLTEXT/5.xml'

In [4]:
from google.colab import drive
drive.mount('/content/drive/content/La_Otra_Alemania')

ValueError: Mountpoint must be in a directory that exists

In [ ]:
print(content)

In [ ]:
print(type(content))

Datei direkt als XML laden:

In [ ]:
xml = BeautifulSoup(open('La_Otra_Alemania/1942-01-01_BWSVNEKFTM7SPW4SQAWQQNGIHHTRFDS4/DDB_FULLTEXT/5.xml'),'lxml-xml')
print(xml.prettify())

In [ ]:
print(type(xml))

#### Laden **aller** .xml-Dateien aus einem Unterordner

Laden aller Dateien aus dem Unterverzeichnis "La_Otra_Alemania/1942-01-01_BWSVNEKFTM7SPW4SQAWQQNGIHHTRFDS4":

In [ ]:
folder_xml = []

for filepath in Path('./La_Otra_Alemania/1942-01-01_BWSVNEKFTM7SPW4SQAWQQNGIHHTRFDS4').glob('*/*.XML'):
    with filepath.open() as f:
        soup = BeautifulSoup(f,'lxml-xml')
        folder_xml.append(soup)

In [ ]:
print(len(folder_xml))

In [ ]:
print(folder_xml)

#### Laden ALLER .xml-Dateien im Unterverzeichnis "La_Otra_Alemania"

Statt einer Liste wird nun ein Dictionary erzeugt, welches zur besseren Nachvollziehbarkeit als Schlüssel auch den jeweiligen Dateinamen enthält:

In [ ]:
all_xml = {}

for filepath in Path('./La_Otra_Alemania').glob('**/*.XML'):
    with filepath.open(encoding='utf-8') as f:
        # read as string:
        xml_string = f.read()
        #Dateiname inklusive der Namen der beiden übergeordneten Ordner als Schlüssel:
        key = f"{filepath.parent.parent.name}/{filepath.parent.name}/{filepath.name}"
        all_xml[key] = xml_string

In [ ]:
print(len(all_xml))

#### Kurze Erkärung zu .glob:

- Mustererkennung: Die glob-Methode wird verwendet, um Dateipfade mit einem bestimmten Muster abzugleichen. In diesem Fall ist das Muster '\*/\*.XML'.
- Musterdetails:
    - '\*' entspricht einer beliebigen Anzahl von Zeichen, einschließlich keinem.
    - '\*.XML' entspricht jeder Datei mit der Erweiterung '.XML'.
    - Das Muster '\*/\*.XML' sucht speziell nach '.XML'-Dateien, die sich eine Verzeichnisebene unterhalb des angegebenen Pfads ('./Folder') befinden.
- Rekursive Suche: Das Muster '\*/\*.XML' sucht nach XML-Dateien in allen unmittelbaren Unterverzeichnissen von './Folder'. Um rekursiv durch alle Unterverzeichnisse in beliebiger Tiefe zu suchen, wird das Muster '\*\*/\*.XML' verwendet.


#### Umwandlung in ein Pandas-Dataframe


In [ ]:
df = pd.DataFrame(list(all_xml.items()), columns=['filename', 'content'])
df

#### Suchen von möglichen Problemen (optional):

In [ ]:
# Sucht nach dem Index von Reihen, in denen die Spalte "text" keinen Inhalt hat:
empty_text_indices = df[df['text'] == ""].index

# Ausgabe der gefundenen Zeilen ohne Text:
for index in empty_text_indices:
    print(f"Index: {index}, Content: {df.loc[index, 'Filename']}")

#### Text aus XML extrahieren:

In [ ]:
def extract_text(content):
    # Remove XML declaration if present
    if content.startswith('<?xml'):
        content = content.split('?>', 1)[1]

    NS = {'alto': 'http://www.loc.gov/standards/alto/ns-v2#'}
    tree = etree.fromstring(unicodedata.normalize("NFC", content))

    text_lines = []  # Initialize as an empty list to store text lines

    for line in tree.xpath('//alto:TextLine', namespaces=NS):
        text = " ".join(
            word for word in line.xpath('alto:String/@CONTENT', namespaces=NS))
        text_lines.append(text)  # Append each extracted text line to the list

    return " ".join(text_lines)  # Return all text as a single string

Erstellen einer neuen Spalte "text". In diese wird das jeweilige Ergebnis der Anwendung der Funktion "extract_text" auf das korrespondierende Element in der Spalte "content" des Dataframes "df" geschrieben:

In [ ]:
df['text'] = df['content'].apply(extract_text)
df

Löschen der Spalte "content" um Speicherplatz zu sparen:

In [ ]:
df2 = df.drop(columns=['content'])
df2

In [ ]:
# Zeilen löschen, in denen die Spalte 'Text' leer ist
df2 = df2[df2['text'].notna() & (df2['text'] != '')]
df2


Speichern als CSV:

In [ ]:
df2.to_csv("otra_alemania_content.csv", encoding = "UTF-8")